# WZUM General Template

## Wczytywanie danych z csv

In [0]:
import pandas as pd
dataset = pd.read_csv('Data.csv')
X = dataset.iloc[:, :-1].values # Wszytskie kolumny bez ostatniej
y = dataset.iloc[:, -1].values # Tylko ostatnia kolumna

print(X.describe()) # Wyświetlenie statystyk zbioru

## Uzupełnienie brakująych danych

In [0]:
from sklearn.impute import SimpleImputer
import numpy as np
imputer = SimpleImputer(missing_values=np.nan, strategy='mean') # Zastępujemy wszystkie wystąpienia NaN wartością średnią
imputer.fit(X[:, 1:3])
X[:, 1:3] = imputer.transform(X[:, 1:3])

## Kodowanie danych tekstowych

### Kodowanie cech

#### Funkcja wbudowana

In [0]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough') # Zamienia wartość tekstową w kolumnie 0 na kombinacje 1.0 0.0 0.0 itd., zawsze wstawia zakodowane wartości na początku
X = np.array(ct.fit_transform(X))

#### Ręczna

In [ ]:
X_train =[[15, 12, "brak"],
          [20, 15, "brak"],
          [10, 3, "małe"],
          [15, 8, "brak"]]
y_train = [1, 1, 1, 0]
opady_d = {"brak": 0, "małe": 1, "średnie": 2, "duże": 3} # Kodujemy ostatnią kolumnę
for i, m in enumerate(X_train):
    m[2] = opady_d[m[2]] # Zamiana wartości

### Kodowanie zmiennej wyjściowej

In [0]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder() # Zamienia dwie wartości np. yes i no na 1.0 i 0.0
y = le.fit_transform(y)

## Podział na zbiór uczący i testowy

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1, stratify=y)

## Skalowanie cech

## Standaryzacja

In [0]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler() # Standaryzacja
X_train[:, 3:] = sc.fit_transform(X_train[:, 3:])
X_test[:, 3:] = sc.transform(X_test[:, 3:])

## Normalizacja

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
sc.fit(X_train)
X_train = sc.transform(X_train) # Normalizujemy wartości od 0 do 1

## Wyniki klasyfikatora

### Score

In [ ]:
from sklearn.svm import SVC
clf = SVC()
clf.fit(X_train, y_train)
train_score = clf.score(X_train, y_train)
print(f'train_score: {train_score}')

### Macierz pomyłek

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
predicted = clf.predict(X_test)
expected = y_test
cm = confusion_matrix(expected, predicted) # Wygenerowanie tablicy pomyłek
print(f'confusion_matrix: \n{cm}') # Wyświetlenie tablicy pomyłek
disp_cm = ConfusionMatrixDisplay(cm, display_labels=np.unique(dataset.target)) # Graficzne wyświetlenie tablicy pomyłek
disp_cm.plot()
plt.show()

# Pipeline

In [ ]:
from sklearn.pipeline import Pipeline
mdl = Pipeline([
        ('standard_scaler', StandardScaler()),
        ('min_max_scaler', MinMaxScaler()),
        ('classifier', SVC())
    ])
mdl.fit(X_train, y_train)
y_pred = mdl.predict(X_test)

## Pipeline rozszerzenie

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
clsf = [
    LinearSVC,
    SVC,
    RandomForestClassifier,
    DecisionTreeClassifier
] # Lista klasyfikatorów do testów
results = dict()
for clf in clsf:
    mdl = Pipeline([
        ('standard_scaler', StandardScaler()),
        ('min_max_scaler', MinMaxScaler()),
        ('classifier', clf()) # Tworzenie obiektu klasyfikatora
    ])
    mdl.fit(X_train, y_train)
    results[str(clf.__name__)] = mdl.score(X_test, y_test)

# Zapisywanie modelu

In [ ]:
import pickle
with open('best_clf.pkl', 'wb') as file:
    pickle.dump(clf.best_estimator_, file) # Zapisanie modelu do pliku
    clf = pickle.load(file) # Wczytanie modelu do zmiennej

# Stacking

In [ ]:
from sklearn.ensemble import StackingClassifier
clf = StackingClassifier([
    ('SVC', SVC()),
    ('RF', RandomForestClassifier()),
    ('KNN', LinearSVC())
], SVC()) # Metapredyktor
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

# Grupowanie (voting)

In [ ]:
from sklearn.ensemble import VotingClassifier
clf = VotingClassifier([
    ('SVC', SVC()),
    ('RF', RandomForestClassifier()),
    ('KNN', LinearSVC())
])
clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

# Grid search

In [ ]:
classifier = SVC(kernel = 'rbf')
from sklearn.model_selection import GridSearchCV
parameters = [{'C': [0.25, 0.5, 0.75, 1], 'kernel': ['linear']},
              {'C': [0.25, 0.5, 0.75, 1], 'kernel': ['rbf'], 'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]}]
grid_search = GridSearchCV(estimator = classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)
grid_search.fit(X_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
print("Best Parameters:", best_parameters)

# k cross validation

In [ ]:
classifier = SVC(kernel = 'rbf')
classifier.fit(X_train, y_train)
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))